In [45]:
import os
import numpy as np
import pandas as pd

In [46]:

file_path_1 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/compustat_profit.csv' # Fundamental data
df = pd.read_csv(file_path_1)
df['datadate'] = pd.to_datetime(df['datadate'])
df['rdq'] = pd.to_datetime(df['rdq'])
df = df.drop(columns=['gvkey', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr','costat'])

df = df.sort_values(by=['tic', 'rdq'])

df.head()

,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq
1243,1981-06-30,AAPL,224.0,1981-07-22,201.262,55.136,0.21,NaN,NaN,NaN,0.0
1244,1981-09-30,AAPL,224.0,1981-10-20,227.104,55.309,0.19,NaN,NaN,NaN,0.0
1245,1981-12-31,AAPL,224.0,1982-01-22,249.160,55.421,0.24,NaN,NaN,NaN,0.0
1246,1982-03-31,AAPL,224.0,1982-04-15,263.140,56.488,0.24,NaN,NaN,NaN,0.0
1247,1982-06-30,AAPL,224.0,1982-07-16,283.437,56.743,0.26,NaN,NaN,NaN,0.0


In [47]:
file_path_2 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/prices_profit.csv' # price data
df2 = pd.read_csv(file_path_2)
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2.drop(columns=['PERMNO', 'OPENPRC'])

df2 = df2.sort_values(by=['TICKER', 'date'])
df2 = df2.rename(columns={'TICKER': 'tic'})

df2['PRC'] = df2['PRC'].abs()
#df2['OPENPRC'] = df2['OPENPRC'].abs()

df2.loc[(df2['tic'] == 'MA') & (df2['date'] == '2014-01-31'), 'PRC'] = 75.68

df2['30_day_MA'] = df2.groupby('tic')['PRC'].rolling(window=30, min_periods=1).mean().reset_index(level=0, drop=True)


df2.head()

,date,tic,PRC,30_day_MA
312596,1980-12-12,AAPL,28.8125,28.812500
312597,1980-12-15,AAPL,27.3125,28.062500
312598,1980-12-16,AAPL,25.3125,27.145833
312599,1980-12-17,AAPL,25.9375,26.843750
312600,1980-12-18,AAPL,26.6875,26.812500


In [48]:
import pandas as pd
df = df.dropna(subset=['rdq'])

# Merge the dataframes on 'tic' and ensure that the price is on or after 'rdq'
df = pd.merge_asof(df.sort_values('rdq'), 
                          df2.sort_values('date'), 
                          by='tic', 
                          left_on='rdq', 
                          right_on='date', 
                          direction='forward')

# Now merged_df will have an extra column with the stock price for the date closest to 'rdq'
df = df.sort_values(by=['tic', 'rdq'])
df.head()


,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq,date,PRC,30_day_MA
1683,1981-06-30,AAPL,224.0,1981-07-22,201.262,55.136,0.21,NaN,NaN,NaN,0.0,1981-07-22,22.7500,27.437500
1726,1981-09-30,AAPL,224.0,1981-10-20,227.104,55.309,0.19,NaN,NaN,NaN,0.0,1981-10-20,19.6875,17.712500
1772,1981-12-31,AAPL,224.0,1982-01-22,249.160,55.421,0.24,NaN,NaN,NaN,0.0,1982-01-22,20.8125,20.397917
1812,1982-03-31,AAPL,224.0,1982-04-15,263.140,56.488,0.24,NaN,NaN,NaN,0.0,1982-04-15,16.4375,16.670833
1864,1982-06-30,AAPL,224.0,1982-07-16,283.437,56.743,0.26,NaN,NaN,NaN,0.0,1982-07-16,13.3125,12.820833


In [49]:
file_path_3 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/s&p500index.csv' # s&p500
df3 = pd.read_csv(file_path_3)
df3['caldt'] = pd.to_datetime(df3['caldt'])  # Adjust column name if needed

df3.tail()

,caldt,spindx
25794,2023-12-22,4754.63
25795,2023-12-26,4774.75
25796,2023-12-27,4781.58
25797,2023-12-28,4783.35
25798,2023-12-29,4769.83


In [50]:
df = df.sort_values(by=['rdq'])


#merge (fill empty cells with nearest s&p500 value)
df = pd.merge_asof(df, df3, left_on='rdq', right_on='caldt', direction='nearest')
df = df.drop(columns=['caldt'])
df = df[df['date'] <= '2023-12-29']

df['PRC'] = df['PRC'] / df['ajexq']
#df['OPENPRC'] = df['OPENPRC'] / df['ajexq']
df['epsfiq'] = df['epsfiq'] / df['ajexq']
df['cshoq'] = df['cshoq'] * df['ajexq']
df['cshfdy'] = df['cshfdy'] * df['ajexq']

df = df.sort_values(by=['tic', 'rdq']).reset_index(drop=True)
df.tail()

,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq,date,PRC,30_day_MA,spindx
15170,2022-09-30,ZTS,1.0,2022-11-03,6551.0,466.442,1.13,4662.0,471.600,1171.0,0.325,2022-11-03,131.14000,149.062332,3719.89
15171,2022-12-31,ZTS,1.0,2023-02-14,7506.0,463.808,0.99,4403.0,470.385,1912.0,0.325,2023-02-14,171.89999,160.982334,4136.13
15172,2023-03-31,ZTS,1.0,2023-05-04,6269.0,462.495,1.19,4491.0,464.600,549.0,0.375,2023-05-04,178.47000,171.414666,4061.22
15173,2023-06-30,ZTS,1.0,2023-08-08,6183.0,460.751,1.45,4621.0,463.800,732.0,0.375,2023-08-08,189.30000,177.123000,4499.38
15174,2023-09-30,ZTS,1.0,2023-11-02,6222.0,459.525,1.29,5074.0,463.000,1456.0,0.375,2023-11-02,160.91000,169.442999,4317.78


In [51]:
# compute ratios
# Use OPENPRC first, if NaN then fall back to PRC

df['ptb'] = (df['30_day_MA'] * df['cshoq']) / df['teqq']
df['div_yield'] = df['dvpspq'] / df['30_day_MA']
df['pe_inc'] = df['epsfiq'] / df['ajexq']
#df['pcf'] = (df['OPENPRC'].fillna(df['PRC']) * df['cshfdy']) / df['oancfy']

df = df.drop(columns=['ajexq', 'date', '30_day_MA', 'cshoq', 'cshfdy', 'oancfy'])


df.tail(10)


,datadate,tic,rdq,actq,epsfiq,teqq,dvpspq,PRC,spindx,ptb,div_yield,pe_inc
15165,2021-06-30,ZTS,2021-08-05,7048.0,1.07,4352.0,0.250,203.84000,4429.10,21.533015,0.001265,1.07
15166,2021-09-30,ZTS,2021-11-04,6682.0,1.16,4681.0,0.250,217.77000,4680.06,20.576280,0.001229,1.16
15167,2021-12-31,ZTS,2022-02-15,6930.0,0.87,4544.0,0.250,197.75999,4471.07,21.121964,0.001231,0.87
15168,2022-03-31,ZTS,2022-05-05,6837.0,1.26,4658.0,0.325,171.45000,4146.87,18.785842,0.001750,1.26
15169,2022-06-30,ZTS,2022-08-04,6651.0,1.12,4580.0,0.325,174.31000,4151.94,17.982052,0.001849,1.12
15170,2022-09-30,ZTS,2022-11-03,6551.0,1.13,4662.0,0.325,131.14000,3719.89,14.913971,0.002180,1.13
15171,2022-12-31,ZTS,2023-02-14,7506.0,0.99,4403.0,0.325,171.89999,4136.13,16.957732,0.002019,0.99
15172,2023-03-31,ZTS,2023-05-04,6269.0,1.19,4491.0,0.375,178.47000,4061.22,17.652733,0.002188,1.19
15173,2023-06-30,ZTS,2023-08-08,6183.0,1.45,4621.0,0.375,189.30000,4499.38,17.660593,0.002117,1.45
15174,2023-09-30,ZTS,2023-11-02,6222.0,1.29,5074.0,0.375,160.91000,4317.78,15.345545,0.002213,1.29


In [52]:


# convert features to pct

features = [
    'actq', 'epsfiq', 'teqq',
    'dvpspq', 'ptb', 'div_yield',
    'pe_inc']

for col in features:
    df[col + '_change'] = df[col].pct_change()


# Calculate the percentage change for 'PRC' within each stock group
df['PRC_change'] = df.groupby('tic')['PRC'].pct_change()
df['next_PRC_change'] = df.groupby('tic')['PRC_change'].shift(-1)

df['spindx_change'] = df.groupby('tic')['spindx'].pct_change()
df['next_spindx_change'] = df.groupby('tic')['spindx_change'].shift(-1)

df['relative_quarterly_return'] = (df['PRC_change'] - df['spindx_change'])
df['next_relative_quarterly_return'] = (df['next_PRC_change'] - df['next_spindx_change'])

#df = df.drop(columns=['prccq', 'spindx'])

df.tail()

,datadate,tic,rdq,actq,epsfiq,teqq,dvpspq,PRC,spindx,ptb,...,dvpspq_change,ptb_change,div_yield_change,pe_inc_change,PRC_change,next_PRC_change,spindx_change,next_spindx_change,relative_quarterly_return,next_relative_quarterly_return
15170,2022-09-30,ZTS,2022-11-03,6551.0,1.13,4662.0,0.325,131.14000,3719.89,14.913971,...,0.000000,-0.170619,0.179024,0.008929,-0.247662,0.310813,-0.104060,0.111896,-0.143602,0.198917
15171,2022-12-31,ZTS,2023-02-14,7506.0,0.99,4403.0,0.325,171.89999,4136.13,16.957732,...,0.000000,0.137037,-0.074045,-0.123894,0.310813,0.038220,0.111896,-0.018111,0.198917,0.056331
15172,2023-03-31,ZTS,2023-05-04,6269.0,1.19,4491.0,0.375,178.47000,4061.22,17.652733,...,0.153846,0.040984,0.083623,0.202020,0.038220,0.060682,-0.018111,0.107889,0.056331,-0.047206
15173,2023-06-30,ZTS,2023-08-08,6183.0,1.45,4621.0,0.375,189.30000,4499.38,17.660593,...,0.000000,0.000445,-0.032228,0.218487,0.060682,-0.149974,0.107889,-0.040361,-0.047206,-0.109612
15174,2023-09-30,ZTS,2023-11-02,6222.0,1.29,5074.0,0.375,160.91000,4317.78,15.345545,...,0.000000,-0.131086,0.045325,-0.110345,-0.149974,NaN,-0.040361,NaN,-0.109612,NaN


In [53]:
# Replace inf and -inf with NaN
df.replace(['inf', '-inf', np.inf, -np.inf], np.nan, inplace=True)

df.head()

,datadate,tic,rdq,actq,epsfiq,teqq,dvpspq,PRC,spindx,ptb,...,dvpspq_change,ptb_change,div_yield_change,pe_inc_change,PRC_change,next_PRC_change,spindx_change,next_spindx_change,relative_quarterly_return,next_relative_quarterly_return
0,1981-06-30,AAPL,1981-07-22,201.262,0.000937,NaN,0.0,0.101562,127.13,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.134615,NaN,-0.053882,NaN,-0.080734
1,1981-09-30,AAPL,1981-10-20,227.104,0.000848,NaN,0.0,0.087891,120.28,NaN,...,NaN,NaN,NaN,-0.095238,-0.134615,0.057143,-0.053882,-0.040738,-0.080734,0.097881
2,1981-12-31,AAPL,1982-01-22,249.160,0.001071,NaN,0.0,0.092913,115.38,NaN,...,NaN,NaN,NaN,0.263158,0.057143,-0.210210,-0.040738,0.008407,0.097881,-0.218617
3,1982-03-31,AAPL,1982-04-15,263.140,0.001071,NaN,0.0,0.073382,116.35,NaN,...,NaN,NaN,NaN,0.000000,-0.210210,-0.190114,0.008407,-0.045380,-0.218617,-0.144734
4,1982-06-30,AAPL,1982-07-16,283.437,0.001161,NaN,0.0,0.059431,111.07,NaN,...,NaN,NaN,NaN,0.083333,-0.190114,0.906103,-0.045380,0.253534,-0.144734,0.652569


In [55]:
df.to_csv('dati_trattati_profit.csv', index=False)  # Set index=False to exclude the index column